In [ ]:
import itertools as it
import operator
import re
import subprocess
import tempfile
import time
from collections import Counter, defaultdict
from functools import partial
from pathlib import Path

import awkward as ak
import duckdb
import gfapy
import holoviews as hv
import ibis
import matplotlib.pyplot as plt
import numba
import numpy as np
import pandas as pd
import parasail
import pod5
import polars as pl
import pyabpoa
import pyarrow as pa
import pyarrow.compute as pc
import pyarrow.dataset as ds
import pyfastx
import pysam
import spoa
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from pyarrow import csv
from pywfa import WavefrontAligner
from tqdm.auto import tqdm, trange

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import paulssonlab.sequencing.align as align
import paulssonlab.sequencing.cigar as scigar
import paulssonlab.sequencing.consensus as con
import paulssonlab.sequencing.gfa as sgfa
import paulssonlab.sequencing.io as sio
import paulssonlab.sequencing.processing as processing
import paulssonlab.sequencing.uuid as uuid
from paulssonlab.util.sequence import reverse_complement

In [ ]:
hv.extension("bokeh")

In [ ]:
%load_ext pyinstrument
import line_profiler
import pyinstrument

%load_ext line_profiler

In [ ]:
pl.enable_string_cache()

# Config

In [ ]:
gfa_filename = "/home/jqs1/scratch3/jqs1/sequencing/230930_alignment_test/barcode.gfa"

In [ ]:
gfa = gfapy.Gfa.from_file(gfa_filename)

# Duplex pairing

In [ ]:
# bam_filename = "/home/jqs1/scratch3/jqs1/sequencing/230930_alignment_test/230707_repressilators/channel-135_merged.bam"
# bam_filename = "/home/jqs1/scratch3/jqs1/sequencing/230922_bcd_rbses_constitutive/20230922_1104_1A_PAQ83451_8d610a8c/bam_pass/channel-100_merged.bam"
bam_filename = "/home/jqs1/scratch/sequencing/230707_repressilators/20230707_2040_MN35044_FAS94231_25542e0d/_temp/channel-1_merged.bam"
gaf_filename = "/home/jqs1/scratch/sequencing/230707_repressilators/20230707_2040_MN35044_FAS94231_25542e0d/_temp/channel-1_merged.gaf"

In [ ]:
# %%time
# bam = pysam.AlignmentFile(bam_filename, check_sq=False)
# bam.reset()
# reads = [{"name": read.query_name, **dict(read.tags)} for read in tqdm(bam.fetch(until_eof=True))]

In [ ]:
# %%time
# reads = pa.Table.from_batches(tqdm(sio.iter_bam_and_gaf(bam_filename, gaf_filename, include_unaligned=False)))

In [ ]:
%%time
df = processing.load_pairing_data(bam_filename, gaf_filename)

In [ ]:
df

In [ ]:
%%time
batches = []
for batch in tqdm(
    sio.iter_bam_and_gaf(bam_filename, gaf_filename, include_unaligned=False)
):
    batch = batch.select(
        ["name", "read_seq", "path", "qs", "mx", "ch", "st", "du", "pi"]
    )
    schema = batch.schema
    schema = schema.set(
        schema.names.index("st"), pa.field("st", pa.timestamp("ms", "UTC"))
    )
    batch = pa.RecordBatch.from_pydict(
        {
            **batch.to_pydict(),
            "st": batch.column("st").cast(pa.timestamp("ms", "UTC")),
        },
        schema=schema,
    )
    batches.append(batch)
df = pl.from_arrow(batches)
df2 = df.with_columns(pl.col("read_seq").str.len_bytes().alias("read_len")).select(
    pl.all().exclude("read_seq")
)

In [ ]:
pl.LazyFrame.

In [ ]:
?pl.concat

In [ ]:
?pl.LazyFrame

In [ ]:
pl.LazyFrame(
    iter(batches),
).tail().collect()  # .with_columns((pl.col("qs") + 1000).alias("read_len")).tail()

In [ ]:
df2

In [ ]:
import networkx as nx

include = []
include_prefix = []
exclude = ["UNS9", "BC:UPSTREAM", "BC:JUNCTION", "BC:T7_TERM", "BC:SPACER2"]
exclude_prefix = []
###
gfa = gfapy.Gfa.from_file(gfa_filename)
gfa = sgfa.filter_gfa(gfa, include, include_prefix, exclude, exclude_prefix)
graph = sgfa.gfa_to_dag(gfa)
# weakly_connected_components is a generator, so only compute once
wccs = list(nx.weakly_connected_components(graph))
forward_segments = sgfa.dag_forward_segments(graph, wccs=wccs)

In [ ]:
(
    path_filter,
    _,
    reverse_path_mapping,
) = processing._segments_for_normalize_path(forward_segments)

In [ ]:
df0 = df

In [ ]:
df = (
    df0.rename({"path": "full_path"})
    # .is_first_distinct() picks better (primary) alignment instead of alternate alignment
    # if input is in GAF order
    # .filter(pl.col("name").is_first_distinct())
    # TODO: polars bug, is_first_distinct() returns empty dataframe
    # when used in a more complicated lazy query,
    # exact cause unknown
    .filter(pl.col("name").is_first_distinct())
    .with_columns(
        pl.col("full_path")
        .list.set_intersection(path_filter)
        # TODO: waiting on https://github.com/pola-rs/polars/issues/11735
        # to keep path columns as categorical
        .cast(pl.List(pl.Categorical))
        .alias("_path"),
    )
    .with_columns(
        pl.col("_path")
        .list.reverse()
        .list.eval(pl.element().replace(reverse_path_mapping, default=None))
        .alias("_path_reversed"),
    )
)
# if not keep_unaligned:
#     df = df.filter(pl.col("path").is_not_null())
# if endpoints:
#     df = df.filter(
#         (pl.col("path").list.set_intersection(endpoints[0]).list.len() > 0)
#         & (pl.col("path").list.set_intersection(endpoints[1]).list.len() > 0)
#     )

In [ ]:
df2 = df  # .filter(pl.col("ch") == 134, pl.col("mx") == 1)
df_sorted = df2.sort("st")
df_endtime = df2.with_columns(
    (pl.col("st") + pl.duration(seconds="du")).alias("_endtime")
).sort("_endtime")

In [ ]:
%%time
df_joined = df_endtime.join_asof(
    df_sorted, left_on="_endtime", right_on="st", strategy="forward", tolerance="10s"
)

In [ ]:
df_joined.filter(pl.col("_path") == pl.col("_path_reversed_right"))

In [ ]:
%%time
df_joined = df_endtime.join_asof(
    df_sorted,
    left_on="_endtime",
    right_on="st",
    by=["ch", "mx"],
    strategy="forward",
    tolerance="10s",
)

In [ ]:
df_joined.filter(pl.col("name_right").is_not_null())

In [ ]:
df4.with_columns((pl.col("st_right") - pl.col("_endtime")).alias("dt"))

In [ ]:
df4.filter(pl.col("name_right").is_not_null())